# sources

## alpaca

In [ ]:
!mkdir -p data/sources/alpaca
!curl -L https://github.com/yizhongw/self-instruct/raw/main/human_eval/user_oriented_instructions.jsonl \
    > data/sources/alpaca/user_oriented_instructions.jsonl

In [99]:
%run -n main.py
items = read_jsonl('data/sources/alpaca/user_oriented_instructions.jsonl')
alpaca_items = list(parse_alpaca(items))
random.sample(alpaca_items, 3)

[{'id': '7610be0f-8237-4035-96a8-5820219d619e',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_224',
  'instruction': 'Convert natural language into a list of turn-by-turn directions.\n\n"Go west on I-10 until you hit the 101 then take it north. You will eventually see the Hollywood sign."'},
 {'id': 'a58bfb9a-4f0d-4330-8154-4afd17d573c6',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_238',
  'instruction': 'Categorize the given product into one of Electronics, Computers, Smart Home, or Arts & Crafts departments.\n\n"Google Nest Learning Thermostat"'},
 {'id': '63ce006e-7056-46cd-842b-3615c2d8317b',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_104',
  'instruction': 'List the personality traits that are required to be successful in the given job.\n\n"Social Media Marketer"'}]

## vicuna

In [ ]:
!mkdir -p data/sources/vicuna
!curl -L https://github.com/lm-sys/vicuna-blog-eval/raw/main/eval/table/question.jsonl \
    > data/sources/vicuna/question.jsonl

In [100]:
%run -n main.py
items = read_jsonl('data/sources/vicuna/question.jsonl')
vicuna_items = list(parse_vicuna(items))
random.sample(vicuna_items, 3)

[{'id': '2aea7930-c299-4be5-aad4-048d669c231d',
  'source': 'vicuna',
  'source_id': 53,
  'category': 'counterfactual',
  'instruction': 'What if the Black Death had not occurred in the 14th century?'},
 {'id': '5b49aa61-1300-495a-ac10-1d88a3186939',
  'source': 'vicuna',
  'source_id': 73,
  'category': 'writing',
  'instruction': 'Use an appropriate format to structure a formal letter of recommendation for a student applying to a prestigious graduate program in computer science.'},
 {'id': '7c08ea72-8c64-4b93-9bde-445f20dbe3b1',
  'source': 'vicuna',
  'source_id': 57,
  'category': 'counterfactual',
  'instruction': 'What if the Suez Canal had never been constructed?'}]

## arena

In [ ]:
!mkdir -p data/sources/arena
!curl -L curl -L https://huggingface.co/datasets/lmsys/chatbot_arena_conversations/resolve/main/data/train-00000-of-00001-cced8514c7ed782a.parquet \
    > data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet

In [101]:
%run -n main.py
records = pd.read_parquet('data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet').itertuples()
instruction_items = {
    _['instruction']: _
    for _ in parse_arena(records)
    if _['lang'] == 'English'
}
arena_items = list(instruction_items.values())
random.sample(arena_items, 3)

[{'id': '0581e7f7-8d1a-4752-bb31-98812c0a3219',
  'source': 'arena',
  'source_id': 'b0432b57c2294f3296e89526b739b0e5',
  'lang': 'English',
  'instruction': 'tell me story synopsis of moble suit gundam'},
 {'id': '4c591123-3a12-43e6-897c-bbc2dff0d131',
  'source': 'arena',
  'source_id': '363a3fa4b50c48ba91c8dce9995caca4',
  'lang': 'English',
  'instruction': "To create a simple notation for representing types of dice rolls, we can use abbreviations, numbers, and operators to clearly convey the steps needed. Here is a notation system:\n\n1. Use 'R' to represent 'roll'.\n2. Numbers preceding 'd' represent the number of dice to roll.\n3. Numbers following 'd' represent the type of dice (e.g., d6 for a six-sided die).\n4. Use 'D' followed by a number to represent 'drop' and specify the number of dice to drop.\n5. Use 'L' or 'H' after the drop command to specify if you're dropping the lowest or highest rolls.\n6. Use '+' to indicate summing the remaining dice.\n\nUsing this notation, her

# translate

In [98]:
import uuid

str(uuid.uuid4())

'0dc748a8-ec42-44bb-b582-77636035c639'

In [37]:
import uu

uu?